In [1]:
# How changing the dataset sample size effects the calibration methods
# Fix training dataset size and change the calib set samples - best method is one that gets max calib with least data

# imports
import sys
import numpy as np
import pandas as pd
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import core_exp as cx
import core_calib as cal
import core_tools as ct

In [2]:
params = {
    # exp
    "seed": 0,
    "runs": 2,
    "exp_name": ct.generate_readable_short_id("rf_large_fix_tree"), #"main_run5_cv5_21data_100trees_40opt_fast",
    "path": "../../",
    "split": "CV", #CV, random_split
    "cv_folds": 5,
    "plot": False,
    "calib_methods": ["RF_d", "RF_opt",
                    #   "Platt", "ISO", "Beta", "CRF", "VA",
                    #   "tlr", "Rank", #"Elkan"
                      # "RF_ens_k", "RF_ens_r", 
                      "RF_large",
                      ],
    
    "metrics": ["acc", "brier", "ece", "logloss"],

    # calib param
    "bin_strategy": "uniform",
    "ece_bins": 20,
    "boot_size": 1000, 
    "boot_count": 5,

    # RF hyper opt
    "hyper_opt": True,
    "opt_cv":5, 
    "opt_n_iter":20,
    "opt_top_K": 5,
    "search_space": {
                    "n_estimators": [30],
                    "max_depth": np.arange(5, 100).tolist(),
                    "criterion": ["gini", "entropy"],
                    "max_features": ['sqrt', 'log2', None],
                    "min_samples_split": np.arange(2, 11).tolist(),
                    "min_samples_leaf": np.arange(1, 11).tolist(),
                    # 'bootstrap': [True, False],                # Whether bootstrap samples are used
                    'class_weight': [None, 'balanced', 'balanced_subsample'],  # Weight for each class
                    "oob_score": [False]
                    },
    
    "oob": False,

    "exp_key": "data_name",
    "exp_values": [
                  "vertebral", 
                  "wilt",
                  "parkinsons", 
                  "heart",
                  "wdbc",
                #   "bank", 
                #   "ionosphere", 
                #   "HRCompetencyScores",
                #   "spambase", 
                #   "QSAR", 
                #   "diabetes", 
                #   "breast", 
                #   "SPF",
                #   "hillvalley",
                #   "pc4",
                #   "scene",
                #   "Sonar_Mine_Rock_Data",
                #   "Customer_Churn",
                #   "jm1",
                #   "eeg",
                  # "madelon",
                  # "phoneme",
                  # "nomao",
                  ]
}

ct.save_params(params)

In [3]:
calib_results_dict, data_list = cx.run_exp(params["exp_key"], params["exp_values"], params)
ct.save_results(calib_results_dict, params['exp_name'])

exp_param vertebral done
exp_param wilt done
exp_param parkinsons done
exp_param heart done
exp_param wdbc done


In [4]:
tables = cal.mean_and_ranking_table(calib_results_dict, 
                                    params["metrics"], 
                                    params["calib_methods"], 
                                    data_list, 
                                    mean_and_rank=True, 
                                    std=True)
ct.save_metrics_to_csv(tables, params["metrics"], params['exp_name'])

In [5]:
tables["brier"].round(4)


,RF_d,RF_opt,RF_large
Data,,,
vertebral,0.1059,0.1054,0.1078
wilt,0.0135,0.0129,0.0129
parkinsons,0.0765,0.0974,0.0892
heart,0.1303,0.1340,0.1329
wdbc,0.0312,0.0332,0.0304
Mean,0.0715,0.0766,0.0747
Rank,1.8000,2.2000,2.0000


In [6]:
cm = [
    "RF_d", "RF_opt",
    "Platt", "ISO", "Beta", "CRF", "VA",
    "tlr", "Rank", 
    "RF_large"
]

ct.res_statistics(tables, params["metrics"], f"results/{params['exp_name']}")

metric acc
Friedman Test Statistic: 2.2105263157894752
P-value: 0.33112373295101094
The differences between groups are not significant.
metric brier
Friedman Test Statistic: 0.4000000000000057
P-value: 0.8187307530779795
The differences between groups are not significant.
metric ece
Friedman Test Statistic: 2.8000000000000043
P-value: 0.24659696394160596
The differences between groups are not significant.
metric logloss
Friedman Test Statistic: 2.8000000000000043
P-value: 0.24659696394160596
The differences between groups are not significant.
